In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("snoplus")

In [ ]:
data = pd.read_csv("cosmic.csv", names=["id","sec","ns","name","pid","process","energy","x","y","z","tank","wbls","death"], delimiter=" ")
tank = data.query('wbls != 1')
data = data.query('wbls == 1')
norm = (data.x**2 + data.y**2 + data.z**2)**0.5
data['direction'] = data.z/norm
seconds = max(data.sec) - min(data.sec)
frequency = len(np.unique(data.id))/seconds
tankrate = len(np.unique(tank.id))/seconds
totalrate = len(np.unique(np.concatenate([data.id, tank.id])))/seconds
#data = pd.read_csv("cosmic.csv")

In [ ]:
plt.plot(data.id, data.sec, label=f'Events (>1 MeV) in\nWbls: {frequency:0.1f} Hz\nTank: {tankrate:0.1f} Hz\nTotal: {totalrate:0.1f} Hz')
plt.xlim(0, 1e7)
plt.ylim(bottom=0)
plt.xlabel("Event ID")
plt.ylabel("Time (seconds)")
plt.legend()
plt.savefig("figures/CosmicEventTimes.svg")
plt.show()

In [ ]:
np.unique(data.process)
michel = (data[data.process == 'muMinusCaptureAtRest'])
michel = michel[michel.name == 'e-']
nmichel = len(michel)
michelFrequency = nmichel / seconds
label = f'Michel electrons\n{nmichel} ({michelFrequency:0.1f} Hz)'

plt.hist(michel.energy, bins=np.arange(0,100,2), label=label, histtype='step')
plt.xlabel("Electron initial energy (MeV)")
plt.ylabel(f"Counts / 1 MeV / {seconds:0.0f} seconds")
plt.xlim(0, 80)
plt.legend()
plt.savefig('figures/Michel.svg')
plt.show()

In [ ]:
# General directionality of everything
maxZ = -np.cos(np.pi/4)
muons = data[data.process=='start']
label = f'All "start" processes'
fractionMissed = sum(muons.direction>maxZ)/len(muons)
plt.hist(muons.direction, bins=np.arange(-1,1,0.02), histtype='step', label=label)
plt.xlim(-1, 1)
plt.xlabel(r"$\cos\theta$")
plt.ylabel(f"Counts / {seconds:0.0f} seconds")
plt.axvline(maxZ, color='xkcd:maroon', label=f'Eos Diagonal\n({fractionMissed*100:0.1f}% cannot hit top and bottom)')
plt.legend()
plt.savefig("figures/AngularDistribution.svg")
plt.show()

In [ ]:
ebins = np.arange(0, 10000, 100)
#fig = plt.figure(figsize=(12, 10))
for particle in np.unique(data.name):
    subset = data[ data.name == particle ]
    #subset = subset[ subset.energy > 10 ]
    if len(subset) > 0:
        rate = len(subset)/seconds
        labelV = f'{rate:0.1f}' if rate > 0.1 else f'{rate:0.1e}'
        plt.hist(subset.energy, bins=ebins, label=f'{particle} ({labelV})', histtype='step')
plt.yscale('log')
plt.xlim(ebins.min(), ebins.max())
plt.title("Legend shows event rates in Hz")
plt.legend(ncol=6, prop={'size':7})
plt.xlabel("Initial energy (MeV)")
plt.ylabel(f"Counts / 100 MeV / {seconds:0.0f} seconds")
plt.savefig("figures/EnergySpectrum.svg")
plt.show()

In [ ]:
# "Trigger" rate as a function of energy
def triggerRate(df, threshold):
    subset = df[df.energy > threshold]
    count = len(np.unique(subset.id))
    return count / seconds

def combinedRate(df1, df2, thresholds):
    newdf = pd.concat([df1, df2])
    return np.array([triggerRate(newdf, t) for t in thresholds])

thresholds = np.arange(0,10,0.1)
wblsrates = np.array([triggerRate(data, t) for t in thresholds])
tankrates = np.array([triggerRate(tank, t) for t in thresholds])
totalrates = combinedRate(data, tank, thresholds)
#totalrates = np.array([triggerRate(pd.concat([data,tank]), t) for t in thresholds])
plt.title = 'Counting events with 1 or more \nparticles in wbls above threshold'
plt.plot(thresholds, wblsrates, label="WbLS")
plt.plot(thresholds, tankrates, label="Water")
plt.plot(thresholds, totalrates, label="Total")
plt.ylabel("Trigger rate (Hz)")
plt.xlabel("Lower energy threshold (MeV)")
plt.xlim(thresholds.min(), thresholds.max())
plt.ylim(bottom=0)
plt.legend()
plt.savefig('figures/LowEThreshold.svg')
plt.show()